In [1]:
import sys
chione_directory = "/Users/rricker/GitLab/projects/chione"
sys.path.append(chione_directory)

from visualization import visualization_tools
from data_handler.sea_ice_concentration_products import SeaIceConcentrationProducts
import xarray as xr
import matplotlib.pyplot as plt
import glob
import re
import os
import datetime


def visu_xarray(x,y,z,figsize,vmin,vmax,n_level,cmap,time_string,label,outfile, iceconc=None):
    crs = ccrs.Stereographic(central_latitude=90.0, central_longitude=-45.0,
                             false_easting=0.0, false_northing=0.0,
                             true_scale_latitude=70.0)

    fig = plt.figure(figsize=figsize)
    xc,yc = np.meshgrid(x,y)
    bounds = list(np.linspace(vmin,vmax,num=n_level))
    norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N, extend='neither')
    ax = plt.subplot(projection=crs)

    ax.set_extent([0, 180, 45, 90])

    if iceconc:
        if len(iceconc)>0:
            iceconc["ice_conc"][iceconc["ice_conc"]<15.0] = np.nan
            iceconc["ice_conc"][iceconc["ice_conc"]>=15.0] = 1.0
            ax.pcolormesh(xc, -yc, iceconc["ice_conc"],cmap='Greys',vmin=1,vmax=2, alpha=0.4)

    im = ax.pcolormesh(xc, yc, z, cmap=cmap, vmin=vmin, vmax=vmax, norm=norm)
    ax.add_feature(cartopy.feature.OCEAN, facecolor=[(0.16,0.22,0.33)])
    ax.add_feature(cartopy.feature.LAND, facecolor=[(0.05,0.07,0.14)],zorder=2)
    ax.coastlines(linewidth=0.15,color='white',zorder=3)
    ax.axis("off")
    cax = ax.inset_axes([0.51, 0.93, 0.45, 0.02], transform=ax.transAxes)
    cb = plt.colorbar(im,ax=ax,shrink=0.7,orientation='horizontal',cax=cax)
    plt.annotate(time_string[6:8]+'.'+time_string[4:6]+'.'+time_string[0:4],
                 xy=(0.51, 0.965),fontsize=12, xycoords='axes fraction', color='white')

    cb.set_label(label=label,size=12,color='white')
    cb.ax.tick_params(labelsize=12,color='white',labelcolor='white')
    cb.outline.set_linewidth(0)
    plt.close(fig)
    fig.savefig(outfile, bbox_inches='tight',pad_inches=0, dpi=120)


def visualization(config):
    sensor = 'cryosat2'#config['options']['sensor']
    target_var = config['options']['target_variable']
    hem = config["options"]["hemisphere"]
    out_epsg = config["options"]["out_epsg"]
    file_list = sorted('/Volumes/data2/projects/chione/product/cryosat2/netcdf_ex2' + "/" + '*.nc')
    file_list =sorted(glob.glob(config["dir"]["netcdf"] + "/" + '*.nc'))

    out_dir = config['dir'][sensor]['visu']
    
    sic_product = SeaIceConcentrationProducts(hem=hem, product_id=config['options']['ice_conc_product'],
                                              out_epsg=out_epsg)
    sic_product.get_file_list(config['dir']['auxiliary']['ice_conc'][config['options']['ice_conc_product']])
    sic_product.get_file_dates()
    figsize = (8,8)
    
    for file in file_list:
        time_str = re.search('nh-(.+?)-(.*).nc', os.path.basename(file)).group(1)
        dt1d = datetime.timedelta(days=1)
        t0 = datetime.datetime.strptime(date_string, '%Y%m%d')
        t1 = t0+dt1d
        
        sic_product.target_files = sic_product.get_target_files(t0, t1)
        ice_conc = sic_product.get_ice_concentration(sic_product.target_files)

        data  = xr.open_dataset(file,decode_times=False)

        if var == 'sea_ice_thickness':
            vmin, vmax, n_level = 0,5,11
            d_vmin, d_vmax, n_level = -2, 2, 11
            cmap = plt.cm.plasma
            d_cmap = plt.cm.bwr
            scaling = 1.0
            label = 'Sea ice thickness in m'
            d_label = 'Total freeboard difference in cm'

        elif var == 'snow_freeboard':
            vmin, vmax, n_level = 0,60,13
            d_vmin, d_vmax, n_level = -20, 20, 11
            cmap = plt.cm.plasma
            d_cmap = plt.cm.bwr
            scaling = 100.0
            label = 'Total freeboard in cm'
            d_label = 'Total freeboard difference in cm'

        elif var == 'snow_freeboard_corrected':
            vmin, vmax, n_level = 0,60,13
            d_vmin, d_vmax, n_level = -15, 15, 13
            cmap = plt.cm.plasma
            d_cmap = plt.cm.bwr #sns.color_palette("vlag", as_cmap=True)
            scaling = 100.0
            label = 'Total freeboard in cm'
            d_label = 'Total freeboard difference in cm'

        elif var == "dist_acquisition":
            vmin, vmax, n_level = 0,160,17
            cmap = plt.cm.plasma
            scaling = 1.0
            label = 'Distance to data aquisition in km'

        elif var== "time_offset_acquisition":
            vmin, vmax, n_level = -15,15,13
            cmap = plt.cm.bwr
            scaling = 1.0
            label = 'time offfset to data aquisition in days'

        elif var == "snow_freeboard_growth":
            vmin, vmax, n_level = -1,1,20
            cmap = plt.cm.bwr
            scaling = 100.0
            label = 'Total freeboard growth in cm/day'

        elif var == "snow_freeboard_growth_interpolated":
            vmin, vmax, n_level = -0.5,0.5,20
            cmap = plt.cm.bwr
            scaling = 100.0
            label = 'Total freeboard growth in cm/day'

        else:
            print("variable not found")
            break

        if not os.path.exists(out_dir + var):
            try:
                os.mkdir(out_dir + var)
            except OSError as error:
                print(error)

        outfile = out_dir + var + os.sep + re.split('.nc', os.path.basename(file))[0] + '_' + var + '.png'
        visu_xarray(data.xc, data.yc, data[var] * scaling,
                    figsize,
                    vmin, vmax, n_level,
                    cmap,
                    time_string,
                    label,
                    outfile,
                    iceconc=ice_conc)

In [2]:
import yaml
with open('/Users/rricker/GitLab/projects/chione/config.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
visualization(config)

5
6
7


/Users/rricker/miniforge3/envs/is2snow-env/lib/python3.8/site-packages/cartopy/crs.py:228: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
/Users/rricker/miniforge3/envs/is2snow-env/lib/python3.8/site-packages/cartopy/crs.py:280: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
/Users/rricker/miniforge3/envs/is2snow-env/lib/python3.8/site-packages/cartopy/crs.py:347: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
/Users/rricke

8
5
6
7
8
5
6
7
8
5
6
7
8


KeyboardInterrupt: 

In [ ]:
sensor = 'cryosat2'
target_var = config['options']['target_variable']
hem = config["options"]["hemisphere"]
out_epsg = config["options"]["out_epsg"]
file_list = sorted(glob.glob(config['dir'][sensor]['netcdf'] + "/" + '*.nc'))
out_dir = config['dir'][sensor]['visu']

sic_product = SeaIceConcentrationProducts(hem=hem, product_id=config['options']['ice_conc_product'],
                                          out_epsg=out_epsg)
sic_product.get_file_list(config['dir']['auxiliary']['ice_conc'][config['options']['ice_conc_product']])


In [38]:
date_string = re.search('nh-(.+?)-(.*).nc', os.path.basename(file_list[0])).group(1)
dt1d = datetime.timedelta(days=1)
t0 = datetime.datetime.strptime(date_string, '%Y%m%d')
t1 = t0+dt1d

In [39]:
t0

datetime.datetime(2019, 10, 1, 0, 0)

In [40]:
t1

datetime.datetime(2019, 10, 2, 0, 0)